In [ ]:
import os
import cv2
import numpy as np
import requests

# API credentials and endpoint
api_key = '6x'
api_endpoint = 'https://api.remove.bg/v1.0/removebg'

# Input and output directories
input_dir = 'profile_image'
output_dir = 'profile_image/bg_rm'

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Function to remove background using image processing
def bg_rm_01(image_path, output_folder):
    
    BLUR = 19
    CANNY_THRESH_1 = 25
    CANNY_THRESH_2 = 255
    MASK_DILATE_ITER = 5
    MASK_ERODE_ITER = 13
    MASK_COLOR = (0.0,0.0,1.0)
    
    img = cv2.imread(image_path)
    # img

    gray = cv2.GaussianBlur(img, (5, 5), 1)
    gray = cv2.cvtColor(gray,cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(gray,cv2.COLOR_BGR2GRAY)

    gray = 255 - cv2.threshold(gray, 0,255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    # gray

    edges = cv2.dilate(gray, kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (21,21)) , iterations = 1)
    edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3)))
    edges = cv2.GaussianBlur(edges, (1, 1), 0)

    contour_info = []
    contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

    for c in contours:
        contour_info.append((
            c,
            cv2.isContourConvex(c),
            cv2.contourArea(c),
        ))
    contour_info = sorted(contour_info, key=lambda c: c[2], reverse=True)
    max_contour = contour_info[0]

    mask = np.zeros(edges.shape)
    cv2.fillConvexPoly(mask, max_contour[0], (255))

    #-- Smooth mask, then blur it --------------------------------------------------------
    # mask = cv2.dilate(mask, None, iterations=MASK_DILATE_ITER)
    mask = cv2.GaussianBlur(mask, (BLUR, BLUR), 0)
    # mask = cv2.erode(mask, None, iterations=MASK_ERODE_ITER)

    #-- Blend masked img into MASK_COLOR background --------------------------------------
    img = img.astype('float32') / 255.0  # for easy blending

    c_red, c_green, c_blue = cv2.split(img)
    img_a = cv2.merge((c_red, c_green, c_blue, mask.astype('float32') / 255.0))

    output_path = os.path.join(output_folder, os.path.splitext(os.path.basename(image_path))[0] + '.png')
    cv2.imwrite(output_path, img_a * 255)
    print(f"Processed: {image_path} -> {output_path}")


# Function to remove background using Remove.bg API
def bg_rm_02(input_path, output_path):
    response = requests.post(
        api_endpoint,
        files={'image_file': open(input_path, 'rb')},
        data={'size': 'auto'},
        headers={'X-Api-Key': api_key}
    )

    if response.status_code == requests.codes.ok:
        with open(output_path, 'wb') as out:
            out.write(response.content)
        print(f"Processed: {input_path} -> {output_path}")
    else:
        print(f"Error for {input_path}: {response.status_code}, {response.text}")


# Function to process images in a directory
def process_images(input_dir, output_dir):
    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                input_path = os.path.join(root, file)
                relative_path = os.path.relpath(input_path, input_dir)
                output_folder = os.path.join(output_dir, os.path.dirname(relative_path))

                # Remove background using both methods
                bg_rm_01(input_path, output_folder)
                bg_rm_02(input_path, output_folder)


# Process images in the input directory
process_images(input_dir, output_dir)

print("Processing complete.")
